In [51]:
import json
import numpy as np
from model_balancing_cvx import ModelBalancingConvex, Q_
from model_balancing import ModelBalancing, Q_
import cvxpy as cp
import pandas as pd
from sbtab import SBtab

In [5]:
with open("../cvxpy/examples/json/three_chain_model_artificial_no_noise_with_noisefree_kinetic_data.json", "rt") as fp:
    data = json.load(fp)

args = {}
args["S"] = np.array(data["network"]["stoichiometric_matrix"])
args["A_act"] = np.array(data["network"]["activation_matrix"])
args["A_inh"] = np.array(data["network"]["inhibition_matrix"])
args["fluxes"] = Q_(data["reaction_fluxes"]["data"]["mean"], data["reaction_fluxes"]["unit"])

args["Keq_gmean"] = Q_(data["kinetic_constants"]["Keq"]["combined"]["geom_mean"], "")
args["Keq_gmean"][2] *= 1000  # fixing standard concentration convension (from 1 mM to 1 M)
args["Keq_ln_cov"] = np.array(data["kinetic_constants"]["Keq"]["combined"]["cov_ln"])

args["kcatf_gmean"] = Q_(data["kinetic_constants"]["Kcatf"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatf"]["unit"])
args["kcatf_ln_cov"] = np.array(data["kinetic_constants"]["Kcatf"]["combined"]["cov_ln"])

args["kcatr_gmean"] = Q_(data["kinetic_constants"]["Kcatr"]["combined"]["geom_mean"], data["kinetic_constants"]["Kcatr"]["unit"])
args["kcatr_ln_cov"] = np.array(data["kinetic_constants"]["Kcatr"]["combined"]["cov_ln"])

args["Km_gmean"] = Q_(data["kinetic_constants"]["KM"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KM"]["unit"])
args["Km_ln_cov"] = np.array(data["kinetic_constants"]["KM"]["combined"]["cov_ln"])

args["Ka_gmean"] = Q_(data["kinetic_constants"]["KA"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KA"]["unit"])
args["Ka_ln_cov"] = np.array(data["kinetic_constants"]["KA"]["combined"]["cov_ln"])

args["Ki_gmean"] = Q_(data["kinetic_constants"]["KI"]["combined"]["geom_mean"], "mM") #data["kinetic_constants"]["KI"]["unit"])
args["Ki_ln_cov"] = np.array(data["kinetic_constants"]["KI"]["combined"]["cov_ln"])

args["conc_met_gmean"] = Q_(data["metabolite_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_met_gstd"] = np.array(data["metabolite_concentrations"]["combined"]["geom_std"])

args["conc_enz_gmean"] = Q_(data["enzyme_concentrations"]["combined"]["geom_mean"], data["metabolite_concentrations"]["unit"])
args["conc_enz_gstd"] = np.array(data["enzyme_concentrations"]["combined"]["geom_std"])

args["rate_law"] = "CM"

In [6]:
mb = ModelBalancingConvex(**args)
mb.initialize_with_gmeans()
print("kcat reverse comparison:")
print("from JSON:  ", args["kcatr_gmean"].m_as("1/s"))
print("calculated: " , np.exp(mb.ln_kcatr.value))

kcat reverse comparison:
from JSON:   [ 97.6   7.2 138.1]
calculated:  [97.6  7.2  0.1]


In [7]:
print("enzyme concentration comparison:")
print("from JSON:  \n", args["conc_enz_gmean"].m_as("M"))
print("calculated: \n" , np.exp(mb.ln_conc_enz.value))

enzyme concentration comparison:
from JSON:  
 [[1.7e-06 1.6e-06 5.0e-07 4.4e-07]
 [1.4e-06 3.3e-06 1.4e-06 1.4e-06]
 [9.4e-07 3.5e-07 1.3e-06 1.2e-06]]
calculated: 
 [[1.7e-06 1.6e-06 5.0e-07 4.4e-07]
 [1.4e-06 3.3e-06 1.4e-06 1.4e-06]
 [5.6e-07 3.2e-07 3.5e-07 2.7e-07]]


In [9]:
%%time

mb = ModelBalancingConvex(**args)

print("Setting enzyme conc. and kcatr gmeans to calculated values...")
# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(f"initial total squared Z-scores = {mb.objective_value:.3f}")
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

Setting enzyme conc. and kcatr gmeans to calculated values...
initial total squared Z-scores = 0.000
optimized total squared Z-scores = 0.000
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 0.0
kcatf = 0.0
kcatr = 0.0
Km = 0.0
Ka = 0.0
Ki = 0.0
CPU times: user 1.31 s, sys: 80.6 ms, total: 1.39 s
Wall time: 1.33 s


In [10]:
%%time

mb = ModelBalancingConvex(**args)
mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")
print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores()

optimized total squared Z-scores = 58.067
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.0
Keq = 11.61
kcatf = 11.61
kcatr = 11.61
Km = 23.23
Ka = 0.0
Ki = 0.0
CPU times: user 1.31 s, sys: 85.5 ms, total: 1.39 s
Wall time: 1.33 s


In [57]:
v = mb.fluxes
c = Q_(np.exp(mb.ln_conc_met.value), "M")
e = Q_(np.exp(mb.ln_conc_enz.value), "M")
g = Q_(-mb.driving_forces.value, "kJ/mol")

In [61]:
sbtabdoc = SBtab.SBtabDocument(name="state_results")

flux_df = pd.DataFrame(v.m_as("mM/s"), columns=["S1", "S2", "S3", "S4"])
flux_df.insert(0, "!QuantityType", "rate of reaction")
flux_df.insert(1, "!Reaction", ["E1", "E2", "E3"])
flux_sbtab = SBtab.SBtabTable.from_data_frame(
    flux_df.astype(str),
    table_id="MetabolicFlux",
    table_name="MetabolicFlux",
    table_type="QuantityMatrix",
    unit="mM/s",
)
sbtabdoc.add_sbtab(flux_sbtab)

conc_met_df = pd.DataFrame(c.m_as("mM"), columns=["S1", "S2", "S3", "S4"])
conc_met_df.insert(0, "!QuantityType", "concentration")
conc_met_df.insert(1, "!Compound", ["X1", "S2", "X3", "X4"])
conc_met_sbtab = SBtab.SBtabTable.from_data_frame(
    conc_met_df.astype(str),
    table_id="MetaboliteConcentrations",
    table_name="Metabolite concentrations",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(conc_met_sbtab)

conc_enz_df = pd.DataFrame(e.m_as("mM"), columns=["S1", "S2", "S3", "S4"])
conc_enz_df.insert(0, "!QuantityType", "concentration of enzyme")
conc_enz_df.insert(1, "!Compound", ["E1", "E2", "E3"])
conc_enz_sbtab = SBtab.SBtabTable.from_data_frame(
    conc_enz_df.astype(str),
    table_id="EnzymeConcentrations",
    table_name="Enzyme concentrations",
    table_type="QuantityMatrix",
    unit="mM",
)
sbtabdoc.add_sbtab(conc_enz_sbtab)

gibbs_energy_df = pd.DataFrame(g.m_as("kJ/mol"), columns=["S1", "S2", "S3", "S4"])
gibbs_energy_df.insert(0, "!QuantityType", "Gibbs energy of reaction")
gibbs_energy_df.insert(1, "!Compound", ["E1", "E2", "E3"])
gibbs_energy_sbtab = SBtab.SBtabTable.from_data_frame(
    gibbs_energy_df.astype(str),
    table_id="ReactionGibbsFreeEnergy",
    table_name="Gibbs free energies of reaction",
    table_type="QuantityMatrix",
    unit="kJ/mol",
)
sbtabdoc.add_sbtab(gibbs_energy_sbtab)

print(sbtabdoc.to_str())
sbtabdoc.write("state_results.tsv");

!!!SBtab DocumentName='state_results' SBtabVersion='1.0' Date='2021-4-2'
!!SBtab TableID='MetabolicFlux' TableType='QuantityMatrix' TableName='MetabolicFlux' SBtabVersion='1.0' Unit='mM/s'
!QuantityType	!Reaction	S1	S2	S3	S4
rate of reaction	E1	0.021387373764911267	0.01460085302465973	0.011737199044558173	0.009199216810779395
rate of reaction	E2	0.021387372521010195	0.014588598605683439	0.011737199044527745	0.009199199058691014
rate of reaction	E3	0.02138737477565732	0.014313621496418424	0.011737199044230595	0.009199219113649869

!!SBtab TableID='MetaboliteConcentrations' TableType='QuantityMatrix' TableName='Metabolite concentrations' SBtabVersion='1.0' Unit='mM'
!QuantityType	!Compound	S1	S2	S3	S4
concentration	X1	6.37611908286746	10.748467799170818	13.831139417702698	10.69817907364608
concentration	S2	0.15999521454625626	0.337407243185946	0.07259644230054538	0.1005756071413424
concentration	X3	1.1233078614881045	4.70311572244766	0.7065758144709312	1.1668577003894767
concentration	X4

In [30]:
mb_noncvx = ModelBalancing(**args)
mb_noncvx.initialize_with_gmeans()

v = mb.fluxes
c = Q_(np.exp(mb_noncvx.ln_conc_met.value), "M")
e = Q_(np.exp(mb_noncvx.ln_conc_enz.value), "M")
g = Q_(mb_noncvx.driving_forces.value, "kJ/mol")

ValueError: not enough values to unpack (expected 2, got 1)